# Setup

In [ ]:
%load_ext autoreload 
# ??? do i need these?
%autoreload 2
from dictionary_learning import CrossCoder # local copy of github.com/mitroitskii/dictionary_learning
from torch.nn.functional import cosine_similarity
import torch as th
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from tqdm.notebook import tqdm
import warnings

th.set_grad_enabled(False)

import sys
sys.path.append("..") # [ ] maybe import stuff via this - eg science_of_finetuning. etc; gotta change .. though

# [ ] figure out which imports are needed and in which order

In [2]:
# [ ] fix up the args

# crosscoder_path = "/share/u/troitskiid/projects/interp-experiments/reasoning_diff_project/checkpoints/L15_mu3.6e-02_lr1e-04_20250305_010218/L15_mu3.6e-02_lr1e-04.pt"
# crosscoder = "L15_mu3.6e-02_lr1e-04"
# crosscoder_path = "/share/u/troitskiid/projects/interp-experiments/reasoning_diff_project/checkpoints/L20-mu3.6e-02-lr1e-04_20250304_163255/L20-mu3.6e-02-lr1e-04.pt"
# crosscoder = "L20-mu3.6e-02-lr1e-04"
crosscoder_path = "/share/u/troitskiid/projects/interp-experiments/reasoning_diff_project/checkpoints/L25_mu3.6e-02_lr1e-04_20250305_010218/L25_mu3.6e-02_lr1e-04.pt"
crosscoder = "L25_mu3.6e-02_lr1e-04"
extra_args = []
exp_name = "eval_crosscoder"
exp_id = ""
device = "cuda"
base_layer = 0
reasoning_layer = 1

In [ ]:
from shutil import rmtree

coder = CrossCoder.from_pretrained(crosscoder_path)
num_layers, activation_dim, dict_size = coder.encoder.weight.shape
print(coder)

save_path = (
    Path("./analysis") / exp_name / (crosscoder + (f"_{exp_id}" if exp_id else ""))
)
if save_path.exists():
    save_path = Path(str(save_path) + "_new")
    if save_path.exists():
        warnings.warn(f"Folder {save_path} already exists, and was removed")
        rmtree(save_path)
save_path_extra = save_path / "extra"
save_path_extra.mkdir(exist_ok=True, parents=True)
save_path_html = save_path / "html"
save_path_html.mkdir(exist_ok=False)
save_path_data = save_path / "data"
save_path_data.mkdir(exist_ok=False)
if device == "auto":
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
else:
    device = th.device(device)
all_info = {}
feature_df = {k: {} for k in range(dict_size)}

<!-- # Latent analysis on test set -->

# NOTE: not using this

In [4]:
# if args.add_activation_stats:
#     stats_path = Path(f"../stats/{crosscoder}")
#     fw_stats = th.load(stats_path / "fineweb.pt").rescaled.joint
#     lmsys_stats = th.load(stats_path / "lmsys.pt").rescaled.joint
#     fw_deads = (fw_stats.non_zero_counts == 0).cpu()
#     fw_avg_activations = (
#         fw_stats.avg_activation
#         * fw_stats.total_tokens
#         / fw_stats.non_zero_counts.float()
#     ).cpu()
#     fw_freqs = (fw_stats.non_zero_counts.float() / fw_stats.total_tokens).cpu()
#     lmsys_deads = (lmsys_stats.non_zero_counts == 0).cpu()
#     lmsys_avg_activations = (
#         (
#             lmsys_stats.avg_activation
#             * lmsys_stats.total_tokens
#             / lmsys_stats.non_zero_counts.float()
#         ).cpu()
#     ).cpu()
#     lmsys_freqs = (lmsys_stats.non_zero_counts.float() / lmsys_stats.total_tokens).cpu()
#     freqs = (
#         (fw_stats.non_zero_counts + lmsys_stats.non_zero_counts).float()
#         / (fw_stats.total_tokens + lmsys_stats.total_tokens).float()
#     ).cpu()
#     deads = fw_deads & lmsys_deads
#     avg_activations = (
#         fw_avg_activations * fw_stats.non_zero_counts.cpu()
#         + lmsys_avg_activations * lmsys_stats.non_zero_counts.cpu()
#     ).float() / (fw_stats.non_zero_counts + lmsys_stats.non_zero_counts).float().cpu()

#     for f_idx, (
#         fw_dead,
#         fw_freq,
#         lmsys_dead,
#         lmsys_freq,
#         freq,
#         dead,
#         avg_activation,
#         lmsys_avg_activation,
#         fw_avg_activation,
#     ) in enumerate(
#         zip(
#             fw_deads,
#             fw_freqs,
#             lmsys_deads,
#             lmsys_freqs,
#             freqs,
#             deads,
#             avg_activations,
#             lmsys_avg_activations,
#             fw_avg_activations,
#         )
#     ):
#         feature_df[f_idx]["fw_dead"] = fw_dead.item()
#         feature_df[f_idx]["fw_freq"] = fw_freq.item()
#         feature_df[f_idx]["lmsys_dead"] = lmsys_dead.item()
#         feature_df[f_idx]["lmsys_freq"] = lmsys_freq.item()
#         feature_df[f_idx]["freq"] = freq.item()
#         feature_df[f_idx]["dead"] = dead.item()
#         feature_df[f_idx]["avg_activation"] = avg_activation.item()
#         feature_df[f_idx]["lmsys_avg_activation"] = lmsys_avg_activation.item()
#         feature_df[f_idx]["fw_avg_activation"] = fw_avg_activation.item()

#     print(f"frac of dead features: {deads.float().mean().item()}")
#     th.save(deads, save_path_data / "deads.pt")
# else:
#     deads = th.zeros(dict_size, dtype=bool)
# alive_mask = ~deads

# Weight analysis

## Compare feature norms

In [ ]:
deads = th.zeros(dict_size, dtype=bool)
alive_mask = ~deads

norms = coder.decoder.weight.norm(dim=-1)
norm_diffs = (
    (norms[base_layer] - norms[reasoning_layer]) / norms.max(dim=0).values + 1
) / 2
norm_diffs = norm_diffs.cpu()
for f_idx, (base_norm, reasoning_norm, norm_diff) in enumerate(
    zip(norms[base_layer], norms[reasoning_layer], norm_diffs)
):
    feature_df[f_idx]["dec_base_norm"] = base_norm.item()
    feature_df[f_idx]["dec_reasoning_norm"] = reasoning_norm.item()
    feature_df[f_idx]["dec_norm_diff"] = norm_diff.item()

# Create color array based on norm difference categories
colors = [
    (
        "green"
        if x <= 0.1
        else "orange" if x >= 0.9 else "blue" if 0.4 <= x <= 0.6 else "grey"
    )
    for x in norm_diffs[alive_mask]
]

# Create masks for each category
reasoning_only_mask = norm_diffs[alive_mask] <= 0.1
base_only_mask = norm_diffs[alive_mask] >= 0.9
shared_mask = (0.4 <= norm_diffs[alive_mask]) & (norm_diffs[alive_mask] <= 0.6)
other_mask = ~(reasoning_only_mask | base_only_mask | shared_mask)

# Create figure
fig = go.Figure()

# Add histogram traces for each category
fig.add_trace(
    go.Histogram(
        x=norm_diffs[alive_mask][reasoning_only_mask],
        name="Reasoning only",
        marker_color="green",
        nbinsx=50,
    )
)

fig.add_trace(
    go.Histogram(
        x=norm_diffs[alive_mask][base_only_mask],
        name="Base only",
        marker_color="orange",
        nbinsx=50,
    )
)

fig.add_trace(
    go.Histogram(
        x=norm_diffs[alive_mask][shared_mask],
        name="Shared",
        marker_color="blue",
        nbinsx=50,
    )
)

fig.add_trace(
    go.Histogram(
        x=norm_diffs[alive_mask][other_mask],
        name="Other",
        marker_color="grey",
        nbinsx=50,
    )
)

fig.update_layout(
    annotations=[
        dict(
            x=0,
            y=0,
            xref="x",
            yref="paper",
            text="  <b>(Reasoning only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
            font=dict(color="green"),
        ),
        dict(
            x=0.5,
            y=0,
            xref="x",
            yref="paper",
            text="    <b>(shared)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
            font=dict(color="blue"),
        ),
        dict(
            x=1,
            y=0,
            xref="x",
            yref="paper",
            text="  <b>(Base only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
            font=dict(color="orange"),
        ),
    ],
    shapes=[
        dict(
            type="line",
            x0=0.1,
            y0=0,
            x1=0.1,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="green", width=2),
        ),
        dict(
            type="line",
            x0=0.9,
            y0=0,
            x1=0.9,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="orange", width=2),
        ),
        dict(
            type="line",
            x0=0.4,
            y0=0,
            x1=0.4,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="blue", width=2),
        ),
        dict(
            type="line",
            x0=0.6,
            y0=0,
            x1=0.6,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="blue", width=2),
        ),
    ],
    showlegend=False,
)
fig.update_xaxes(tickvals=[0, 0.5, 1])
# fig.update_traces(hovertemplate='Feature Index: %{text}<br>Value: %{y}', text=sorted_norm_diffs.indices)
fig.update_xaxes(title=f"Sorted Features (Highest to Lowest Difference) for {crosscoder}")
fig.update_yaxes(title="Relative Decoder Feature Norm Difference")
fig.update_layout(barmode="stack")
fig.show()
fig.write_html(save_path_html / "decoder_norm_diffs.html")
fig.write_image(save_path / "decoder_norm_diffs.png")

In [ ]:
fig = px.histogram(
    x=norm_diffs[deads],
    title="Relative difference in decoder dead feature norms",
    orientation="v",
    nbins=50,
)
fig.update_layout(
    annotations=[
        dict(
            x=0,
            y=0,
            xref="x",
            yref="paper",
            text="  <b>(Reasoning only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
        ),
        dict(
            x=0.5,
            y=0,
            xref="x",
            yref="paper",
            text="    <b>(shared)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
        ),
        dict(
            x=1,
            y=0,
            xref="x",
            yref="paper",
            text="  <b>(Base only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
        ),
    ]
)
fig.update_xaxes(tickvals=[0, 0.5, 1])
# fig.update_traces(hovertemplate='Feature Index: %{text}<br>Value: %{y}', text=sorted_norm_diffs.indices)
fig.update_xaxes(title="Sorted Features (Highest to Lowest Difference)")
fig.update_yaxes(title="Relative Norm Difference")
fig.show()
fig.write_html(save_path_html / "decoder_norm_diffs_dead.html")
fig.write_image(save_path_extra / "decoder_norm_diffs_dead.png")

In [7]:
# save Chat only and base only feature index
treshold = 0.1
only_reasoning_feature_indices = th.nonzero(norm_diffs < treshold, as_tuple=True)[0]
only_base_feature_indices = th.nonzero(norm_diffs > 1 - treshold, as_tuple=True)[0]
shared_feature_indices = th.nonzero((norm_diffs - 0.5).abs() < treshold, as_tuple=True)[
    0
]

th.save(only_reasoning_feature_indices, save_path_data / "only_reasoning_decoder_feature_indices.pt")
th.save(
    only_base_feature_indices, save_path_data / "only_base_decoder_feature_indices.pt"
)
th.save(shared_feature_indices, save_path_data / "shared_decoder_feature_indices.pt")
all_info["frac of dead Reasoning only features"] = (
    deads[only_reasoning_feature_indices].float().mean().item()
)
all_info["frac of dead base only features"] = (
    deads[only_base_feature_indices].float().mean().item()
)
all_info["frac of dead shared features"] = (
    deads[shared_feature_indices].float().mean().item()
)
is_other_feature = th.ones_like(deads, dtype=bool)
is_other_feature[only_reasoning_feature_indices] = False
is_other_feature[only_base_feature_indices] = False
is_other_feature[shared_feature_indices] = False
all_info["frac of dead other features"] = deads[is_other_feature].float().mean().item()

for f_idx in only_reasoning_feature_indices.tolist():
    feature_df[f_idx]["tag"] = "Reasoning only"
for f_idx in only_base_feature_indices.tolist():
    feature_df[f_idx]["tag"] = "Base only"
for f_idx in shared_feature_indices.tolist():
    feature_df[f_idx]["tag"] = "Shared"
for f_idx in is_other_feature.nonzero(as_tuple=True)[0].tolist():
    feature_df[f_idx]["tag"] = "Other"

In [ ]:
enc_norms = coder.encoder.weight.norm(dim=1)
enc_norm_diffs = (
    (enc_norms[base_layer] - enc_norms[reasoning_layer]) / enc_norms.max(dim=0).values
    + 1
) / 2
enc_norm_diffs = enc_norm_diffs.cpu()

for f_idx, (base_norm, reasoning_norm, norm_diff) in enumerate(
    zip(enc_norms[base_layer], enc_norms[reasoning_layer], enc_norm_diffs)
):
    feature_df[f_idx]["enc_base_norm"] = base_norm.item()
    feature_df[f_idx]["enc_reasoning_norm"] = reasoning_norm.item()
    feature_df[f_idx]["enc_norm_diff"] = norm_diff.item()

fig = px.histogram(
    x=enc_norm_diffs[alive_mask],
    title="Relative difference in encoder alive feature norms",
    orientation="v",
    nbins=50,
)
fig.update_xaxes(title="Relative Norm Difference")
fig.update_yaxes(title="Count")
fig.update_layout(
    annotations=[
        dict(
            x=0,
            y=0,
            xref="x",
            yref="paper",
            text="<b>(Reasoning only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
        ),
        dict(
            x=0.5,
            y=0,
            xref="x",
            yref="paper",
            text="<b>(shared)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
        ),
        dict(
            x=1,
            y=0,
            xref="x",
            yref="paper",
            text="<b>(Base only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
        ),
    ]
)
fig.show()
fig.write_html(save_path_html / "encoder_norm_diffs.html")
fig.write_image(save_path / "encoder_norm_diffs.png")

In [ ]:
enc_norms = coder.encoder.weight.norm(dim=1)
enc_norm_diffs = (
    (enc_norms[base_layer] - enc_norms[reasoning_layer]) / enc_norms.max(dim=0).values
    + 1
) / 2

for f_idx, (base_norm, reasoning_norm, norm_diff) in enumerate(
    zip(enc_norms[base_layer], enc_norms[reasoning_layer], enc_norm_diffs)
):
    feature_df[f_idx]["enc_base_norm"] = base_norm.item()
    feature_df[f_idx]["enc_reasoning_norm"] = reasoning_norm.item()
    feature_df[f_idx]["enc_norm_diff"] = norm_diff.item()

fig = px.histogram(
    x=enc_norm_diffs[deads],
    title="Relative difference in encoder dead feature norms",
    orientation="v",
    nbins=50,
)
fig.update_xaxes(title="Relative Norm Difference")
fig.update_yaxes(title="Count")
fig.update_layout(
    annotations=[
        dict(
            x=0,
            y=0,
            xref="x",
            yref="paper",
            text="<b>(Reasoning only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
        ),
        dict(
            x=0.5,
            y=0,
            xref="x",
            yref="paper",
            text="<b>(shared)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
        ),
        dict(
            x=1,
            y=0,
            xref="x",
            yref="paper",
            text="<b>(Base only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
        ),
    ]
)
fig.show()
fig.write_html(save_path_html / "encoder_norm_diffs_dead.html")
fig.write_image(save_path_extra / "encoder_norm_diffs_dead.png")

## Cosine similarity

### Decoder

In [10]:
decoder_cos_sims = cosine_similarity(
    coder.decoder.weight[base_layer], coder.decoder.weight[reasoning_layer], dim=1
)
for f_idx, cos_sim in enumerate(decoder_cos_sims):
    feature_df[f_idx]["dec_cos_sim"] = cos_sim.item()

decoder_cos_sims_sorted = decoder_cos_sims.sort(descending=True)

# Calculate mean cosine similarity between random latent vectors
num_samples = 10000
random_idx = th.randint(0, coder.decoder.weight.shape[1], (num_samples, 2))
random_cos_sims = cosine_similarity(
    coder.decoder.weight[base_layer][random_idx[:, 0]],
    coder.decoder.weight[reasoning_layer][random_idx[:, 1]],
    dim=1,
)
mean_random_cos_sim = random_cos_sims.mean().item()

In [ ]:
fig = px.histogram(
    x=decoder_cos_sims[alive_mask].cpu(),
    title="Cosine similarity between decoder alive feature vectors",
    orientation="v",
    nbins=50,
)
fig.update_xaxes(title="Cosine Similarity")
fig.update_yaxes(title="Count")
fig.add_vline(
    x=mean_random_cos_sim,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean Random Cosine Similarity: {mean_random_cos_sim:.4f}",
)
fig.show()
fig.write_html(save_path_html / "decoder_cos_sims.html")
fig.write_image(save_path / "decoder_cos_sims.png")

In [ ]:
import pandas as pd

df = pd.DataFrame(feature_df).T

filtered_decoder_cos_sims = decoder_cos_sims[alive_mask].cpu()
filtered_tags = df["tag"][alive_mask.cpu().numpy()]

# Normalize by amount of each tag that are not dead
tag_counts = filtered_tags.value_counts()

# Create histogram with normalized counts
fig = px.histogram(
    x=filtered_decoder_cos_sims,
    title="Cosine similarity between decoder feature vectors (Alive)",
    orientation="v",
    nbins=50,
    color=filtered_tags,
    color_discrete_map={
        "Shared": "blue",
        "Base only": "orange",
        "Reasoning only": "green",
        "Other": "grey",
    },
    # Normalize histogram counts by tag frequency
    histnorm="probability",
    # Group by tag to normalize each category separately
    barmode="group",
)

fig.update_xaxes(title="Cosine Similarity")
fig.update_yaxes(title="Normalized Count", range=[0, 1])
fig.add_vline(
    x=mean_random_cos_sim,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean Random Cosine Similarity: {mean_random_cos_sim:.4f}",
)
fig.show()
fig.write_html(save_path_html / "decoder_cos_sims_colored.html")
fig.write_image(save_path / "decoder_cos_sims_colored.png")

In [ ]:
fig = px.histogram(
    x=decoder_cos_sims[deads],
    title="Cosine similarity between decoder dead feature vectors",
    orientation="v",
    nbins=50,
)
fig.update_xaxes(title="Cosine Similarity")
fig.update_yaxes(title="Count")
fig.add_vline(
    x=mean_random_cos_sim,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean Random Cosine Similarity: {mean_random_cos_sim:.4f}",
)
fig.show()
fig.write_html(save_path_html / "decoder_cos_sims_dead.html")
fig.write_image(save_path_extra / "decoder_cos_sims_dead.png")

### Encoder

In [14]:
encoder_cos_sims = cosine_similarity(
    coder.encoder.weight[base_layer], coder.encoder.weight[reasoning_layer], dim=0
)
for f_idx, cos_sim in enumerate(encoder_cos_sims):
    feature_df[f_idx]["enc_cos_sim"] = cos_sim.item()

mean_random_cos_sim_encoder = (
    cosine_similarity(
        coder.encoder.weight[base_layer][:, random_idx[:, 0]],
        coder.encoder.weight[reasoning_layer][:, random_idx[:, 1]],
        dim=0,
    )
    .mean()
    .item()
)

In [ ]:
fig = px.histogram(
    x=encoder_cos_sims[alive_mask].cpu(),
    title="Cosine similarity between encoder alive feature vectors",
    orientation="v",
    nbins=50,
)
fig.update_xaxes(title="Cosine Similarity")
fig.update_yaxes(title="Count")
fig.add_vline(
    x=mean_random_cos_sim_encoder,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean Random Cosine Similarity: {mean_random_cos_sim_encoder:.4f}",
)
fig.show()
fig.write_html(save_path_html / "encoder_cos_sims.html")
fig.write_image(save_path / "encoder_cos_sims.png")

In [ ]:
fig = px.histogram(
    x=encoder_cos_sims[deads],
    title="Cosine similarity between encoder dead feature vectors",
    orientation="v",
    nbins=50,
)
fig.update_xaxes(title="Cosine Similarity")
fig.update_yaxes(title="Count")
fig.add_vline(
    x=mean_random_cos_sim_encoder,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Mean Random Cosine Similarity: {mean_random_cos_sim_encoder:.4f}",
)
fig.show()
fig.write_html(save_path_html / "encoder_cos_sims_dead.html")
fig.write_image(save_path_extra / "encoder_cos_sims_dead.png")

# End

In [20]:
import pandas as pd
import json

pd.DataFrame(feature_df).T.to_csv(save_path_data / "feature_df.csv")
with open(save_path_data / "all_info.json", "w") as f:
    json.dump(all_info, f)

In [21]:
# if stem ends with  _new
# rm the old folder
# remove _new from name
# Check if the save_path stem ends with '_new'
if str(save_path).endswith("_new"):
    # Remove the old folder
    old_folder = Path(str(save_path).removesuffix("_new"))  # Remove '_new' from the end
    if old_folder.exists():
        rmtree(old_folder)

    # Rename the new folder by removing '_new'
    save_path.rename(old_folder)
    save_path = old_folder  # Update the save_path variable

In [ ]:
df = pd.DataFrame(feature_df).T
display(df.sort_values(by="dec_norm_diff", ascending=True))